# Introdução

Conforme ensinado pelo professor Federson na disciplina de **PAD**, utilizei o método **AGEMC** para a construção da linha de raciocínio que responde à atividade proposta. O método AGEMC é um framework que estrutura a análise em cinco etapas: **Ask a Question**, **Get the Data**, **Explore Data**, **Modeling** e **Communicate**.

1. **Ask a Question**: Definimos a pergunta principal da análise para orientar todas as etapas subsequentes.
2. **Get the Data**: Coletamos e carregamos o conjunto de dados necessário para responder à pergunta.
3. **Explore Data**: Analisamos e pré-processamos os dados, tratando valores ausentes e identificando as características dos atributos.
4. **Modeling**: Construímos e avaliamos um modelo, neste caso, uma Árvore de Decisão, aplicando diferentes métodos de avaliação para comparar seus impactos na acurácia.
5. **Communicate**: Apresentamos os resultados e conclusões de maneira clara, respondendo à pergunta inicial com base nos achados da análise.

A atividade proposta requer a comparação de três métodos de avaliação para o modelo de classificação: **Hold-out**, **10-fold Cross-Validation** e **Teste Externo**. O objetivo é observar como cada método afeta a acurácia do modelo ao classificar a faixa de renda no dataset **Adult**.

O uso do AGEMC organiza o processo de análise de forma estruturada, facilitando tanto a execução quanto a comunicação dos resultados obtidos.

# A - Ask a Question

## Pergunta
**"Como os diferentes métodos de avaliação impactam a acurácia de um modelo de Árvore de Decisão ao prever a faixa de renda no dataset Adult?"**

A atividade visa comparar três métodos de avaliação (Hold-out, 10-fold Cross-Validation e Teste Externo) para entender suas diferenças em termos de acurácia ao classificar a renda como >50K ou <=50K. O objetivo é determinar qual método fornece uma estimativa mais confiável do desempenho do modelo.


# G - Get the Data

## Coleta e Carregamento dos Dados
Nesta etapa, carregamos os dados necessários para a análise. O dataset **Adult** está dividido em dois arquivos: `adult.data` (para treino) e `adult.test` (para teste externo). 


In [1]:
# Importar as dependências necessárias ao longo de todo o projeto.
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
# Carregar os dados dos arquivos 'adult.data' e 'adult.test'.
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
    'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
    'hours_per_week', 'native_country', 'income'
]

# Carregar o conjunto de treinamento e de teste.
train_data = pd.read_csv(f"adult/adult.data", names=columns, na_values=" ?", skipinitialspace=True)
test_data = pd.read_csv(f"adult/adult.test", names=columns, na_values=" ?", skipinitialspace=True, skiprows=1)

# Visualizar as primeiras linhas para confirmar o carregamento correto.
display(train_data)
display(test_data)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
16277,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K.
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


# E - Explore Data

## Preparação e Exploração dos Dados
Nesta etapa, fazemos a separação das variáveis independentes e dependentes, identificamos colunas numéricas e categóricas e preparamos o pipeline de pré-processamento para tratar valores ausentes e variáveis categóricas.

In [3]:
# Preparação dos dados.
X_train = train_data.drop("income", axis=1)
y_train = train_data["income"].apply(lambda x: 1 if x.strip() == ">50K" else 0)
X_test = test_data.drop("income", axis=1)
y_test = test_data["income"].apply(lambda x: 1 if x.strip() == ">50K." else 0)

In [4]:
# Colunas numéricas e categóricas.
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

# M - Modeling

## Treinamento e Avaliação
Nesta seção, construímos e avaliamos o modelo de Árvore de Decisão usando três métodos de avaliação: Hold-out, 10-fold Cross-Validation e Teste Externo. Cada método utiliza a acurácia como métrica.


In [5]:
# Transformadores para pré-processamento.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Pré-processamento completo.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [6]:
# Pipeline com Árvore de Decisão.
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', DecisionTreeClassifier(random_state=42))])

In [7]:
# Função auxiliar para exibir a acurácia média.
def evaluate_model(pipeline, X, y, method, test_data=None, test_labels=None):
    if method == "Hold-out":
        X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X, y, test_size=1/3, random_state=42)
        pipeline.fit(X_train_split, y_train_split)
        y_pred = pipeline.predict(X_val_split)
        accuracy = accuracy_score(y_val_split, y_pred)
    elif method == "10-fold CV":
        scores = cross_val_score(pipeline, X, y, cv=10, scoring='accuracy')
        accuracy = scores.mean()
    elif method == "External Test":
        pipeline.fit(X, y)
        y_pred = pipeline.predict(test_data)
        accuracy = accuracy_score(test_labels, y_pred)
    return accuracy

In [8]:
# Avaliação para cada método.
holdout_accuracy = evaluate_model(pipeline, X_train, y_train, method="Hold-out")
cv_accuracy = evaluate_model(pipeline, X_train, y_train, method="10-fold CV")
external_test_accuracy = evaluate_model(pipeline, X_train, y_train, method="External Test", test_data=X_test, test_labels=y_test)

In [9]:
# Monstrando os resultados obtidos.
print(f"Hold-out Accuracy: {holdout_accuracy}")
print(f"10-fold CV Accuracy: {cv_accuracy}")
print(f"External Test Accuracy: {external_test_accuracy}")

Hold-out Accuracy: 0.8137092316196793
10-fold CV Accuracy: 0.8193853684258965
External Test Accuracy: 0.8105153246115103



---

# C - Communicate

## Resultados e Conclusão

A pergunta inicial foi: **"Como os diferentes métodos de avaliação impactam a acurácia de um modelo de Árvore de Decisão ao prever a faixa de renda no dataset Adult?"**

Com base nos resultados obtidos, as acurácias de cada método foram:

| Método de Avaliação       | Acurácia (%) |
|---------------------------|--------------|
| Hold-out                  | 81.37        |
| 10-fold Cross-Validation  | 81.94        |
| Teste Externo (`adult.test`) | 81.05    |

### Conclusão
Os resultados mostram que a **validação cruzada 10-fold** forneceu a melhor acurácia, seguida pela divisão **Hold-out** e, por último, o **teste externo** com dados do `adult.test`. A pequena variação entre os métodos sugere que o modelo é relativamente estável, mas a **10-fold Cross-Validation** parece oferecer a estimativa mais confiável do desempenho real, pois aproveita ao máximo os dados disponíveis para gerar um desempenho médio.

Esta comparação entre os métodos permite concluir que a escolha do método de avaliação pode afetar ligeiramente as estimativas de acurácia, mas a 10-fold Cross-Validation, em particular, pode ser preferida em cenários onde se busca maximizar a utilização dos dados e obter uma média mais robusta do desempenho do modelo.

---

Esta análise responde à pergunta inicial e esclarece o impacto de cada método de avaliação no resultado do modelo.

---